In [7]:
import multiprocessing as mp
import mag_functions as F
import requests, json
import time
import re

In [9]:
import requests
import re
from collections import defaultdict, deque

patents = {}

def verify_tags(tag_ids):
    # tag_tree --> {orig_tag: [immediate possible ancestors (id, display_name)]}
    tag_tree = {}
    
    #total_tags --> {tag_id: #, ...}
    total_tags = {}
    for tag_id in tag_ids:
        tag_tree[tag_id] = set()
        total_tags[tag_id] = 1
        
        #ancestors and related concepts
        query = f"https://api.openalex.org/concepts/C{tag_id}?select=ancestors,level,related_concepts"
        response = requests.get(query).json()
        
        if response['level'] != 0:
            immediate_level = response['level'] - 1
            for ancestor in response['ancestors']:
                if ancestor['level'] == immediate_level:
                    ancestor_id = re.sub(r"\D", "", ancestor["id"])
                    
                    tag_tree[tag_id].add(ancestor_id)
                    
                    total_tags[ancestor_id] = total_tags.get(ancestor_id, 0) + 1
        
        for related in response["related_concepts"]:
            related_id = re.sub(r"\D", "", related["id"])
            
            if related_id not in tag_tree[tag_id]:
                tag_tree[tag_id].add(related_id)

                total_tags[related_id] = total_tags.get(related_id, 0) + 1
        
    remaining_results = {
        "tags": [],
        "tag_ids": []
    }
    print("finished checking ancestors and similar tags")
    
    for tag_id in tag_ids:
        query = f"https://api.openalex.org/concepts/C{tag_id}?select=display_name"
        display_name = requests.get(query).json()["display_name"]
        if total_tags[tag_id] > 1:
            remaining_results["tag_ids"].append(tag_id)
            remaining_results["tags"].append(display_name)
        else:
            for rel_anc in tag_tree[tag_id]:
                if total_tags[rel_anc] > 2:
                    remaining_results["tag_ids"].append(tag_id)
                    remaining_results["tags"].append(display_name)
                    break
            
    return remaining_results
        

with open('patentViewJSON.json', 'r') as f:
    data = json.loads(f.read())
    
    for each_author in data:
        if each_author['patents'] is not None:
            for each_patent in each_author['patents']:
                information = {
                    'title': each_patent["patent_title"],
                    'doc_type': None,
                    'journal': None,
                    'abstract': each_patent["patent_abstract"],
                    "inverted_abstract": False
                }
                
                try:
                    start_time = time.time()
                    results = F.get_tags([information], 1)[1][0]
                    results['time'] = time.time() - start_time
                except:
                    results = {"tags": None, "scores": None, "tag_ids": None}
                    print(f"{each_patent['patent_number']} failed")
                
                information['results'] = results
                
                print("finished getting results")
                
                information["remaining_results"] = verify_tags(results["tag_ids"])
                
                patents[each_patent['patent_number']] = information
            

finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting res

finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags
finished getting results
finished checking ancestors and similar tags


In [10]:
with open('patent_results_notype.json', 'w') as f:
    json.dump(patents, f, indent=4)